In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation,BatchNormalization,Lambda,Input,Conv2D,Bidirectional,LSTM, Concatenate, MaxPooling2D,GlobalAveragePooling2D
import tensorflow.keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import time
import datetime
import numpy as np
from generater import crnnGenerator
import logging

logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:

def crnn():
    x=image_input=Input(name='image_input',shape=(128,64,1),dtype='float32')
    """
    minx=Input(name='minx',shape=[1],dtype='float32')
    miny=Input(name='miny',shape=[1],dtype='float32')
    maxx=Input(name='maxx',shape=[1],dtype='float32')
    maxy=Input(name='maxy',shape=[1],dtype='float32')
    def cropimage(img,minx=0,miny=0,maxx=0,maxy=0):
        
        return tf.image.crop_to_bounding_box(img,minx,miny,maxx-minx,maxy-miny)
    x=Lambda(cropimage)([x,minx,miny,maxx,maxy])
    
    """
        
    x=Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_1')(x)#128 64 64
    x=MaxPooling2D(pool_size=(2, 2), name='pool1', padding='same')(x)#64 32 64
    
    x=Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_1')(x)#64 32 128
    x=MaxPooling2D(pool_size=(2, 2), name='pool2', padding='same')(x)# 32 16 128

    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_1')(x)#32 16 256
    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_2')(x)#32 16 256
    x=MaxPooling2D(pool_size=(1, 2), name='pool3', padding='same')(x)# 32 8 256
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_1')(x)#32 8 512
    x=BatchNormalization(name='batch1')(x)
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_1')(x)#32 8 512
    
    x=BatchNormalization(name='batch2')(x)
    x=MaxPooling2D(pool_size=(1, 2), name='pool5')(x)# 32 4 512
    
    x=aa=Conv2D(512, (2, 2), strides=(1, 1), activation='relu', padding='same', name='conv6_1')(x)# 32 4 512
    x=keras.layers.Reshape(target_shape=(32,2048),name='reshape')(x)#32 2048
    x=keras.layers.Dense(64,activation='relu',name='dense1')(x)#32 64
    print(aa)
    x=Bidirectional(LSTM(256, return_sequences=True))(x)
    x=Bidirectional(LSTM(256, return_sequences=True))(x)#32 512
    
    num_classes=36+1
    x=keras.layers.Dense(num_classes, name='dense2')(x)#알파벳+숫자#1953 36
    
    x=y_pred= Activation('softmax', name='softmax')(x)#1953 36
    
   # model_pred=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy],outputs=x)
    model_pred=keras.models.Model(inputs=image_input,outputs=x)
    
    maxstringlen=int(y_pred.shape[1])
    #maxstringlen=100
    
    def ctc_lambda_func(args):
        y_pred,labels, input_length, label_length = args
          #y_pred = y_pred[:, 2:, :] 
        return K.ctc_batch_cost(y_true=labels,y_pred=y_pred,input_length=input_length,label_length=label_length)    
        #return tf.nn.ctc_loss(labels=labels,logits=y_pred,logit_length=input_length,label_length=label_length,logits_time_major=False)    
    
    labels = Input(name='labels', shape=[maxstringlen], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64') 
    
    ctcloss=Lambda(ctc_lambda_func,output_shape=(1,),name='ctcloss')([y_pred,labels,input_length,label_length])
    
   # model_train=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy,labels,input_length,label_length],outputs=ctc_loss)
    model_train=keras.models.Model(inputs=[image_input,labels,input_length,label_length],outputs=ctcloss)
    
    return model_train,model_pred,maxstringlen

    

In [3]:
traincrnn,predcrnn,inputlength=crnn()

KerasTensor(type_spec=TensorSpec(shape=(None, 32, 4, 512), dtype=tf.float32, name=None), name='conv6_1/Relu:0', description="created by layer 'conv6_1'")


In [4]:
predcrnn.load_weights('D:/engocr/checkpoints/202107182100_crnn_train_v1/weights.003.h5')

In [5]:
predcrnn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 128, 64, 64)       640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 64, 32, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 64, 32, 128)       73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 32, 16, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 32, 16, 256)       295168    
_________________________________________________________________
conv3_2 (Conv2D)             (None, 32, 16, 256)       590080

In [6]:
import cv2

In [32]:
imgw=128
imgh=64

img=cv2.imread('D:/engocr/abcd/'+'000adfe5b817011c_11.jpg',cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(imgw,imgh))

img=img.astype(np.float32)
img=(img / 255.0)
#cv2.imshow('ada',img)
#cv2.waitKey()
#cv2.destroyAllWindows()

#imgs[0,:,:]=img
img=img.T
img=np.expand_dims(img,axis=-1)
img=np.expand_dims(img,axis=0)


In [33]:
b=predcrnn.predict(img)

In [9]:
len(imgs[0,:,:])

128

In [10]:
len(img)

128

In [32]:
inputlength=np.ones((32,1))*(128//2)
inputlength

array([[64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.]])

In [26]:
def labels_to_text(labels):     
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    return ''.join(list(map(lambda x: alphabet[int(x)], labels)))

In [28]:
labels_to_text(b[0])

TypeError: only size-1 arrays can be converted to Python scalars

In [34]:
b[0][0]

array([0.04680976, 0.01513381, 0.02014629, 0.01371977, 0.03333002,
       0.01379239, 0.01124316, 0.01857329, 0.03182839, 0.01147583,
       0.0109255 , 0.01439246, 0.02108818, 0.02467737, 0.03622718,
       0.01172854, 0.00697642, 0.03036331, 0.02669594, 0.03060271,
       0.01368243, 0.00920168, 0.00847765, 0.00794493, 0.01073758,
       0.00473634, 0.01230866, 0.00517081, 0.01032454, 0.00864541,
       0.00961741, 0.00620769, 0.00700869, 0.00711093, 0.01142603,
       0.00935825, 0.42831066], dtype=float32)

In [35]:
res=[]
for data in b[0]:
    listdata=list(data)
    res.append( listdata.index(max(listdata)))
res
resu=[]
for i,abc in enumerate(res) :
    if abc!=36:
        resu.append(abc)
res

[36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 36]

In [25]:
ydata=np.ones([8,30])

In [26]:
ydata

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1